# **Running the baseline**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/baseline.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>  

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [ ]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout Camille

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 691, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 691 (delta 183), reused 161 (delta 90), pack-reused 419 (from 1)
Receiving objects: 100% (691/691), 1.04 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (462/462), done.
/content/BornToOverfit
Branch 'Camille' set up to track remote branch 'Camille' from 'origin'.
Switched to a new branch 'Camille'


Installing requirements

In [ ]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 78.3 MB/s eta 0:00:00


Downloading the data

In [ ]:
!cp -r /content/drive/MyDrive/Kaggle_ALTEGRAD/data /content/BornToOverfit/data_baseline/.

cp: cannot open '/content/drive/MyDrive/Kaggle_ALTEGRAD/data/results/baseline.gsheet' for reading: Operation not supported


In [ ]:

import os
import torch
import subprocess
from pathlib import Path

# 1. Configuration des chemins
DATA_DIR = Path("data_baseline/data")

TRAIN_FILE = DATA_DIR / "train_embeddings_RealChemBERT.pt"
VAL_FILE = DATA_DIR / "validation_embeddings_RealChemBERT.pt"

# Fonction pour charger le .pt
def load_embeddings_pt(filepath):
    print(f"📥 Chargement de {filepath}...")

    # On charge le dictionnaire binaire directement
    # map_location='cpu' évite les erreurs si le fichier a été sauvé sur GPU et qu'on charge sur CPU
    data = torch.load(filepath, map_location='cpu')

    # On récupère les clés qu'on a définies lors de la génération
    ids = data['ids']
    embeddings_tensor = data['embeddings']

    print(f"✅ Chargé ! Shape: {embeddings_tensor.shape}")
    return ids, embeddings_tensor

# 2. Logique Principale
if TRAIN_FILE.exists() and VAL_FILE.exists():
    print("🚀 Fichiers .pt trouvés ! Chargement direct...")

    train_ids, train_embeddings = load_embeddings_pt(TRAIN_FILE)
    val_ids, val_embeddings = load_embeddings_pt(VAL_FILE)

    # Exemple d'accès pour vérifier
    print(f"Premier ID train : {train_ids[0]}")
    print(f"Norme du premier vecteur : {train_embeddings[0].norm().item():.4f}")

else:
    print(f"⚠️ Fichiers non trouvés : {TRAIN_FILE}")
    print("⚙️ Lancement du script de génération...")

    cmd = [
        "python", "data_baseline/generate_description_embeddings_automatic.py",
        "--data_dir", "data_baseline/data",
        "--model_name", "recobo/chemical-bert-uncased",
        "--pooling", "cls",
        "--splits", "train", "validation"
    ]

    process = subprocess.run(cmd, capture_output=False)

    if process.returncode == 0:
        print("\n✅ Génération terminée avec succès !")
    else:
        print("\n❌ Erreur lors de la génération.")

🚀 Fichiers .pt trouvés ! Chargement direct...
📥 Chargement de data_baseline/data/train_embeddings_RealChemBERT.pt...
✅ Chargé ! Shape: torch.Size([31008, 768])
📥 Chargement de data_baseline/data/validation_embeddings_RealChemBERT.pt...
✅ Chargé ! Shape: torch.Size([1000, 768])
Premier ID train : 0
Norme du premier vecteur : 24.6847


## **Dual Encoder**

#Code qui va tourner pour train le dual encoder


In [ ]:
%%writefile data_baseline/train_dual_encoder_optimized.py
import os
import argparse
import pickle
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR

from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer

# --- DATASET ---
class RawTextGraphDataset(Dataset):
    def __init__(self, pkl_path):
        self.pkl_path = Path(pkl_path)
        with open(self.pkl_path, 'rb') as f:
            self.data_list = pickle.load(f)
    def __len__(self): return len(self.data_list)
    def __getitem__(self, idx):
        data = self.data_list[idx]
        text = data.description if hasattr(data, 'description') else ""
        return data, text

class DualCollate:
    def __init__(self, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __call__(self, batch):
        graphs, texts = zip(*batch)
        batched_graphs = Batch.from_data_list(graphs)
        text_inputs = self.tokenizer(list(texts), padding=True, truncation=True,
                                     max_length=self.max_len, return_tensors="pt")
        return batched_graphs, text_inputs

# --- MODELS ---
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward(self, batch_graphs, text_inputs):
        g_emb = self.graph_encoder(batch_graphs)
        t_out = self.text_encoder(**text_inputs)
        t_emb = self.text_proj(t_out.last_hidden_state[:, 0, :])
        return F.normalize(g_emb, dim=-1), F.normalize(t_emb, dim=-1)

# --- LOSS CORRIGÉE POUR FP16 ---
def triplet_loss(mol_vec, txt_vec, margin=0.2):
    sims = mol_vec @ txt_vec.t()
    mask = torch.eye(sims.size(0), device=sims.device).bool()

    # CORRECTION : On utilise la plus petite valeur supportée par le type de données actuel
    # En Float16 (AMP), min_value sera ~ -65000, ce qui ne plante pas.
    min_value = torch.finfo(sims.dtype).min
    neg_sims = sims.masked_fill(mask, min_value)

    hard_neg_m2t = neg_sims.max(dim=1, keepdim=True)[0]
    hard_neg_t2m = neg_sims.max(dim=0, keepdim=True)[0].t()
    pos_sims = sims.diag().unsqueeze(1)

    loss = (F.relu(margin + hard_neg_m2t - pos_sims).mean() +
            F.relu(margin + hard_neg_t2m - pos_sims).mean())
    return loss / 2

# --- EVALUATION ---
@torch.no_grad()
def evaluate_retrieval(model, val_loader, device):
    """Évalue le MRR sur le validation set."""
    model.eval()
    all_g_emb, all_t_emb = [], []

    for graphs, text_inputs in val_loader:
        graphs = graphs.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

        # En eval, on peut désactiver autocast ou le laisser, mais ici on le garde
        # pour être cohérent, même si moins critique.
        with autocast():
             g_emb, t_emb = model(graphs, text_inputs)

        all_g_emb.append(g_emb.float()) # On repasse en float32 pour le calcul de similarité précis
        all_t_emb.append(t_emb.float())

    all_g_emb = torch.cat(all_g_emb, dim=0)
    all_t_emb = torch.cat(all_t_emb, dim=0)

    # Text-to-Molecule retrieval
    sims = all_t_emb @ all_g_emb.t()  # [N, N]
    ranks = sims.argsort(dim=-1, descending=True)

    N = sims.size(0)
    correct = torch.arange(N, device=sims.device)
    positions = (ranks == correct.unsqueeze(1)).nonzero()[:, 1] + 1

    mrr = (1.0 / positions.float()).mean().item()
    r1 = (positions <= 1).float().mean().item()
    r5 = (positions <= 5).float().mean().item()
    r10 = (positions <= 10).float().mean().item()

    return {'MRR': mrr, 'R@1': r1, 'R@5': r5, 'R@10': r10}

# --- MAIN ---
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='data_baseline/data')
    parser.add_argument('--model_name', type=str, default='recobo/chemical-bert-uncased')

    # Paramètres à varier
    parser.add_argument('--lr_gnn', type=float, default=8e-4)
    parser.add_argument('--lr_bert', type=float, default=3e-5)
    parser.add_argument('--weight_decay', type=float, default=1e-4)
    parser.add_argument('--freeze_layers', type=int, default=0)
    parser.add_argument('--margin', type=float, default=0.2)

    # Training
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--grad_accum', type=int, default=8)
    parser.add_argument('--epochs', type=int, default=150)
    parser.add_argument('--patience', type=int, default=15)

    args = parser.parse_args()
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    print("=" * 70)
    print("🚀 DUAL ENCODER TRAINING - OPTIMIZED (AMP FIX)")
    print("=" * 70)
    print(f"LR GNN      : {args.lr_gnn}")
    print(f"LR BERT     : {args.lr_bert}")
    print(f"Weight Decay: {args.weight_decay}")
    print(f"Freeze Layers: {args.freeze_layers}")
    print(f"Margin      : {args.margin}")
    print(f"Batch Size  : {args.batch_size} × {args.grad_accum} = {args.batch_size * args.grad_accum} (effective)")
    print("=" * 70)

    tokenizer = AutoTokenizer.from_pretrained(args.model_name)

    # Train loader
    train_loader = DataLoader(
        RawTextGraphDataset(Path(args.data_dir) / "train_graphs.pkl"),
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=DualCollate(tokenizer),
        num_workers=2,
        pin_memory=True
    )

    # Validation loader
    val_loader = DataLoader(
        RawTextGraphDataset(Path(args.data_dir) / "validation_graphs.pkl"),
        batch_size=64,  # Plus gros batch pour eval
        shuffle=False,
        collate_fn=DualCollate(tokenizer),
        num_workers=2,
        pin_memory=True
    )

    gnn_config = {
        'hidden_dim': 256,
        'out_dim': 768,
        'num_layers': 4,
        'num_heads': 4,
        'dropout': 0.1
    }
    model = DualEncoder(args.model_name, gnn_config, freeze_layers=args.freeze_layers).to(DEVICE)

    # Optimizer avec weight decay différencié
    optimizer = torch.optim.AdamW([
        {'params': model.graph_encoder.parameters(),
         'lr': args.lr_gnn,
         'weight_decay': args.weight_decay},
        {'params': model.text_encoder.parameters(),
         'lr': args.lr_bert,
         'weight_decay': args.weight_decay / 10},
        {'params': model.text_proj.parameters(),
         'lr': args.lr_bert,
         'weight_decay': args.weight_decay}
    ])

    scheduler = OneCycleLR(
        optimizer,
        max_lr=[args.lr_gnn, args.lr_bert, args.lr_bert],
        steps_per_epoch=len(train_loader) // args.grad_accum,
        epochs=args.epochs,
        pct_start=0.1
    )

    scaler = GradScaler()
    best_mrr = 0.0
    patience_counter = 0

    for epoch in range(args.epochs):
        # === TRAINING ===
        model.train()
        total_loss = 0
        optimizer.zero_grad()

        for step, (graphs, text_inputs) in enumerate(train_loader):
            graphs = graphs.to(DEVICE)
            text_inputs = {k: v.to(DEVICE) for k, v in text_inputs.items()}

            with autocast():
                g_emb, t_emb = model(graphs, text_inputs)
                loss = triplet_loss(g_emb, t_emb, margin=args.margin) / args.grad_accum

            scaler.scale(loss).backward()

            if (step + 1) % args.grad_accum == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            total_loss += loss.item() * args.grad_accum

        avg_loss = total_loss / len(train_loader)

        # === VALIDATION ===
        val_metrics = evaluate_retrieval(model, val_loader, DEVICE)

        print(f"Epoch {epoch+1}/{args.epochs} | "
              f"Train Loss: {avg_loss:.4f} | "
              f"Val MRR: {val_metrics['MRR']:.4f} | "
              f"R@1: {val_metrics['R@1']:.4f}")

        # === EARLY STOPPING ===
        if val_metrics['MRR'] > best_mrr:
            best_mrr = val_metrics['MRR']
            patience_counter = 0

            effective_bs = args.batch_size * args.grad_accum
            save_name = (f"dual_lrGNN{args.lr_gnn}_lrBERT{args.lr_bert}_"
                        f"wd{args.weight_decay}_frz{args.freeze_layers}_"
                        f"margin{args.margin}_bs{effective_bs}.pt")

            torch.save({
                'model_state_dict': model.state_dict(),
                'args': vars(args),
                'best_mrr': best_mrr
            }, Path(args.data_dir) / save_name)

            print(f"  💾 New Best MRR: {best_mrr:.4f} | Saved: {save_name}")
        else:
            patience_counter += 1
            if patience_counter >= args.patience:
                print("🛑 Early stopping")
                break

if __name__ == "__main__":
    main()

Writing data_baseline/train_dual_encoder_optimized.py


In [ ]:
import subprocess
import pandas as pd
from datetime import datetime
from pathlib import Path
import time
import shutil
from google.colab import drive

# =========================================================
# 0. CONFIGURATION DRIVE & DOSSIERS
# =========================================================
print("🔄 Connexion au Google Drive...")
drive.mount('/content/drive')

# Définissez ici le dossier de VOTRE Drive où vous voulez stocker les résultats
# (Le script créera le dossier s'il n'existe pas)
DRIVE_OUTPUT_DIR = Path("/content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder")
DRIVE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Dossier de sauvegarde sécurisé : {DRIVE_OUTPUT_DIR}")

# Le CSV de résultats sera mis à jour en direct sur le Drive
RESULTS_CSV = DRIVE_OUTPUT_DIR / "dual_encoder_results.csv"

# =========================================================
# CONFIGURATION DES EXPÉRIENCES
# =========================================================
EXPERIMENTS = [
    # === GROUP 1 : LR BERT SWEEP (ROI ⭐⭐⭐⭐⭐) ===
    ## déjà fait #{"lr_gnn": 8e-4, "lr_bert": 1e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Low LR BERT"},
    ## déjà fait #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Baseline BERT"},
    ##à run ##{"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "High LR BERT"},

    # === GROUP 2 : FREEZE STRATEGIES (ROI ⭐⭐⭐⭐) ===
    #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 6, "wd": 1e-4, "margin": 0.2, "name": "Freeze 6 Layers"},
    #{"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 6, "wd": 1e-4, "margin": 0.2, "name": "Freeze 6 + High LR"},
    #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 9, "wd": 1e-4, "margin": 0.2, "name": "Freeze 9 Layers"},

    # === GROUP 3 : WEIGHT DECAY (ROI ⭐⭐⭐) ===
    #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-5, "margin": 0.2, "name": "Low WD"},
    #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 5e-4, "margin": 0.2, "name": "Medium WD"},
    #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-3, "margin": 0.2, "name": "High WD"},

    # === GROUP 4 : MARGIN (ROI ⭐⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.15, "name": "Low Margin"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.25, "name": "High Margin"},

    # === GROUP 5 : LR GNN (ROI ⭐⭐⭐) ===
    {"lr_gnn": 5e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Low LR GNN"},
    {"lr_gnn": 1e-3, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "High LR GNN"},

    # === GROUP 6 : COMBINATIONS (ROI ⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 0, "wd": 5e-4, "margin": 0.25, "name": "Aggressive Full"},
    {"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 6, "wd": 1e-4, "margin": 0.25, "name": "Aggressive Freeze"},
    {"lr_gnn": 5e-4, "lr_bert": 1e-5, "freeze": 0, "wd": 1e-5, "margin": 0.15, "name": "Conservative Full"},

    # === BATCH SIZE VARIATIONS (ROI ⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "batch_size": 32, "grad_accum": 4, "name": "Batch 128 (32x4)"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "batch_size": 8, "grad_accum": 16, "name": "Batch 128 (8x16)"},
]

# Paramètres fixes
FIXED_PARAMS = {
    "data_dir": "data_baseline/data",
    "model_name": "recobo/chemical-bert-uncased",
    "batch_size": 16,
    "grad_accum": 8,
    "epochs": 100,
    "patience": 10,
}

# Chemin du script d'entraînement (local)
SCRIPT_PATH = "data_baseline/train_dual_encoder_optimized.py"

# =========================================================
# AFFICHAGE RÉCAPITULATIF
# =========================================================
print("=" * 80)
print("🚀 DUAL ENCODER - GRID SEARCH AUTOMATIQUE (DRIVE SAVE MODE)")
print("=" * 80)
print(f"Dossier de sortie    : {DRIVE_OUTPUT_DIR}")
print(f"Fichier de résultats : {RESULTS_CSV}")
print(f"Nombre d'expériences : {len(EXPERIMENTS)}")
print("-" * 80)

input("\n⚠️  Appuie sur ENTER pour lancer (ou Ctrl+C pour annuler)...\n")

# Charge les résultats existants si on reprend le run
results = []
if RESULTS_CSV.exists():
    print(f"🔄 Reprise du fichier existant : {RESULTS_CSV}")
    try:
        existing_df = pd.read_csv(RESULTS_CSV)
        results = existing_df.to_dict('records')
        # On saute les runs déjà faits ? (Optionnel, ici on refait tout par sécurité ou on peut filtrer)
        # runs_done = [r['Name'] for r in results]
    except:
        print("⚠️ Impossible de lire le fichier existant, on repart à zéro.")

total_start = datetime.now()

for i, exp in enumerate(EXPERIMENTS, 1):

    # Vérifie si l'expérience a déjà été faite (Optionnel)
    # if exp['name'] in [r.get('Name') for r in results]:
    #     print(f"⏩ Run {i} ({exp['name']}) déjà fait. Skipping...")
    #     continue

    print("\n" + "=" * 80)
    print(f"RUN {i}/{len(EXPERIMENTS)} : {exp['name']}")
    print("=" * 80)
    print(f"LR GNN      : {exp['lr_gnn']}")
    print(f"LR BERT     : {exp['lr_bert']}")
    print(f"Freeze      : {exp['freeze']} layers")
    print(f"Weight Decay: {exp['wd']}")
    print(f"Margin      : {exp['margin']}")
    print("=" * 80)

    # Commande d'entraînement (Les fichiers temp sont créés en local d'abord pour la vitesse)
    cmd = [
        'python', '-u', SCRIPT_PATH,
        '--data_dir', FIXED_PARAMS['data_dir'],
        '--model_name', FIXED_PARAMS['model_name'],
        '--lr_gnn', str(exp['lr_gnn']),
        '--lr_bert', str(exp['lr_bert']),
        '--freeze_layers', str(exp['freeze']),
        '--weight_decay', str(exp['wd']),
        '--margin', str(exp['margin']),
        '--batch_size', str(exp.get('batch_size', FIXED_PARAMS['batch_size'])),
        '--grad_accum', str(exp.get('grad_accum', FIXED_PARAMS['grad_accum'])),
        '--epochs', str(FIXED_PARAMS['epochs']),
        '--patience', str(FIXED_PARAMS['patience']),
    ]

    run_start = datetime.now()
    generated_model_file = None # Pour stocker le nom du fichier créé

    try:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )

        output_lines = []
        for line in process.stdout:
            print(line, end='')
            output_lines.append(line)
            # On guette le nom du fichier sauvegardé en local
            if "Saved: " in line:
                # Ex: "Saved: dual_lrGNN...pt"
                try:
                    generated_model_file = line.split("Saved: ")[1].strip()
                except:
                    pass

        process.wait()
        return_code = process.returncode

        # Parsing des résultats (MRR, etc.)
        best_mrr = 0.0
        best_r1 = 0.0
        best_r5 = 0.0

        for line in output_lines:
            if "New Best MRR:" in line:
                try:
                    mrr_str = line.split("New Best MRR:")[1].split("|")[0].strip()
                    best_mrr = float(mrr_str)
                except: pass
            elif "Val MRR:" in line and "R@1:" in line:
                try:
                    parts = line.split("|")
                    for part in parts:
                        if "Val MRR:" in part: best_mrr = max(best_mrr, float(part.split(":")[1].strip()))
                        elif "R@1:" in part: best_r1 = max(best_r1, float(part.split(":")[1].strip()))
                        elif "R@5:" in part: best_r5 = max(best_r5, float(part.split(":")[1].strip()))
                except: pass

        elapsed = (datetime.now() - run_start).total_seconds() / 60
        status = "✅ SUCCESS" if return_code == 0 else "❌ FAILED"

        # === SAUVEGARDE SUR DRIVE ===
        # Si un fichier .pt a été créé, on le déplace sur le Drive
        if generated_model_file and return_code == 0:
            local_path = Path(FIXED_PARAMS['data_dir']) / generated_model_file
            if local_path.exists():
                drive_path = DRIVE_OUTPUT_DIR / generated_model_file
                print(f"📦 Copie vers Drive : {drive_path} ...")
                shutil.copy2(local_path, drive_path)
                print("✅ Copie terminée.")
            else:
                print(f"⚠️ Fichier local introuvable : {local_path}")

    except Exception as e:
        print(f"\n❌ ERREUR : {e}")
        elapsed = (datetime.now() - run_start).total_seconds() / 60
        status = f"❌ ERROR: {str(e)[:50]}"
        return_code = -1

    # Enregistrement des résultats
    result = {
        'Run': i,
        'Name': exp['name'],
        'LR_GNN': exp['lr_gnn'],
        'LR_BERT': exp['lr_bert'],
        'Freeze_Layers': exp['freeze'],
        'Weight_Decay': exp['wd'],
        'Margin': exp['margin'],
        'MRR': best_mrr,
        'R@1': best_r1,
        'R@5': best_r5,
        'Time_min': elapsed,
        'Status': status,
        'Model_File': generated_model_file if generated_model_file else "N/A"
    }

    # Mise à jour de la liste et sauvegarde CSV immédiate sur le Drive
    results.append(result)
    df_temp = pd.DataFrame(results)
    df_temp.to_csv(RESULTS_CSV, index=False)
    print(f"\n💾 CSV mis à jour sur Drive : {RESULTS_CSV}")

    # Progression
    total_elapsed = (datetime.now() - total_start).total_seconds() / 60
    print(f"\n⏱️  Temps écoulé : {total_elapsed:.1f}min")

    if i < len(EXPERIMENTS):
        print("\n⏸️  Pause de 30s...")
        time.sleep(30)

print("\n🎉 GRID SEARCH TERMINÉ - TOUT EST SUR LE DRIVE !")

🔄 Connexion au Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dossier de sauvegarde sécurisé : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder
🚀 DUAL ENCODER - GRID SEARCH AUTOMATIQUE (DRIVE SAVE MODE)
Dossier de sortie    : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder
Fichier de résultats : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder/dual_encoder_results.csv
Nombre d'expériences : 9
--------------------------------------------------------------------------------

⚠️  Appuie sur ENTER pour lancer (ou Ctrl+C pour annuler)...

🔄 Reprise du fichier existant : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder/dual_encoder_results.csv

RUN 1/9 : Low Margin
LR GNN      : 0.0008
LR BERT     : 3e-05
Freeze      : 0 layers
Weight Decay: 0.0001
Margin      : 0.15
🚀 DUAL ENCODER TRAINING - OPTIMIZED (AMP FIX)
LR GNN      : 0.0008
LR B

KeyboardInterrupt: 

#**⚠️Lancer en local ensuite le run de : *good_conversion_dual.py* pour que le fichier .pt généré soit compatible avec le pipeline de RAPH**

In [ ]:
import subprocess
import pandas as pd
from datetime import datetime
from pathlib import Path
import time
import shutil
from google.colab import drive

# =========================================================
# 0. CONFIGURATION DRIVE & DOSSIERS
# =========================================================
print("🔄 Connexion au Google Drive...")
drive.mount('/content/drive')

# Définissez ici le dossier de VOTRE Drive où vous voulez stocker les résultats
# (Le script créera le dossier s'il n'existe pas)
DRIVE_OUTPUT_DIR = Path("/content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder")
DRIVE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Dossier de sauvegarde sécurisé : {DRIVE_OUTPUT_DIR}")

# Le CSV de résultats sera mis à jour en direct sur le Drive
RESULTS_CSV = DRIVE_OUTPUT_DIR / "dual_encoder_results.csv"

# =========================================================
# CONFIGURATION DES EXPÉRIENCES
# =========================================================
EXPERIMENTS = [
    # === GROUP 1 : LR BERT SWEEP (ROI ⭐⭐⭐⭐⭐) ===
    ## déjà fait #{"lr_gnn": 8e-4, "lr_bert": 1e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Low LR BERT"},
    ## déjà fait #{"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Baseline BERT"},
    ##à run ##{"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "High LR BERT"},

    # === GROUP 2 : FREEZE STRATEGIES (ROI ⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 6, "wd": 1e-4, "margin": 0.2, "name": "Freeze 6 Layers"},
    {"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 6, "wd": 1e-4, "margin": 0.2, "name": "Freeze 6 + High LR"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 9, "wd": 1e-4, "margin": 0.2, "name": "Freeze 9 Layers"},

    # === GROUP 3 : WEIGHT DECAY (ROI ⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-5, "margin": 0.2, "name": "Low WD"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 5e-4, "margin": 0.2, "name": "Medium WD"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-3, "margin": 0.2, "name": "High WD"},

    # === GROUP 4 : MARGIN (ROI ⭐⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.15, "name": "Low Margin"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.25, "name": "High Margin"},

    # === GROUP 5 : LR GNN (ROI ⭐⭐⭐) ===
    {"lr_gnn": 5e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "Low LR GNN"},
    {"lr_gnn": 1e-3, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "name": "High LR GNN"},

    # === GROUP 6 : COMBINATIONS (ROI ⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 0, "wd": 5e-4, "margin": 0.25, "name": "Aggressive Full"},
    {"lr_gnn": 8e-4, "lr_bert": 5e-5, "freeze": 6, "wd": 1e-4, "margin": 0.25, "name": "Aggressive Freeze"},
    {"lr_gnn": 5e-4, "lr_bert": 1e-5, "freeze": 0, "wd": 1e-5, "margin": 0.15, "name": "Conservative Full"},

    # === BATCH SIZE VARIATIONS (ROI ⭐⭐⭐⭐) ===
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "batch_size": 32, "grad_accum": 4, "name": "Batch 128 (32x4)"},
    {"lr_gnn": 8e-4, "lr_bert": 3e-5, "freeze": 0, "wd": 1e-4, "margin": 0.2, "batch_size": 8, "grad_accum": 16, "name": "Batch 128 (8x16)"},
]

# Paramètres fixes
FIXED_PARAMS = {
    "data_dir": "data_baseline/data",
    "model_name": "recobo/chemical-bert-uncased",
    "batch_size": 16,
    "grad_accum": 8,
    "epochs": 100,
    "patience": 10,
}

# Chemin du script d'entraînement (local)
SCRIPT_PATH = "data_baseline/train_dual_encoder_optimized.py"

# =========================================================
# AFFICHAGE RÉCAPITULATIF
# =========================================================
print("=" * 80)
print("🚀 DUAL ENCODER - GRID SEARCH AUTOMATIQUE (DRIVE SAVE MODE)")
print("=" * 80)
print(f"Dossier de sortie    : {DRIVE_OUTPUT_DIR}")
print(f"Fichier de résultats : {RESULTS_CSV}")
print(f"Nombre d'expériences : {len(EXPERIMENTS)}")
print("-" * 80)

input("\n⚠️  Appuie sur ENTER pour lancer (ou Ctrl+C pour annuler)...\n")

# Charge les résultats existants si on reprend le run
results = []
if RESULTS_CSV.exists():
    print(f"🔄 Reprise du fichier existant : {RESULTS_CSV}")
    try:
        existing_df = pd.read_csv(RESULTS_CSV)
        results = existing_df.to_dict('records')
        # On saute les runs déjà faits ? (Optionnel, ici on refait tout par sécurité ou on peut filtrer)
        # runs_done = [r['Name'] for r in results]
    except:
        print("⚠️ Impossible de lire le fichier existant, on repart à zéro.")

total_start = datetime.now()

for i, exp in enumerate(EXPERIMENTS, 1):

    # Vérifie si l'expérience a déjà été faite (Optionnel)
    # if exp['name'] in [r.get('Name') for r in results]:
    #     print(f"⏩ Run {i} ({exp['name']}) déjà fait. Skipping...")
    #     continue

    print("\n" + "=" * 80)
    print(f"RUN {i}/{len(EXPERIMENTS)} : {exp['name']}")
    print("=" * 80)
    print(f"LR GNN      : {exp['lr_gnn']}")
    print(f"LR BERT     : {exp['lr_bert']}")
    print(f"Freeze      : {exp['freeze']} layers")
    print(f"Weight Decay: {exp['wd']}")
    print(f"Margin      : {exp['margin']}")
    print("=" * 80)

    # Commande d'entraînement (Les fichiers temp sont créés en local d'abord pour la vitesse)
    cmd = [
        'python', '-u', SCRIPT_PATH,
        '--data_dir', FIXED_PARAMS['data_dir'],
        '--model_name', FIXED_PARAMS['model_name'],
        '--lr_gnn', str(exp['lr_gnn']),
        '--lr_bert', str(exp['lr_bert']),
        '--freeze_layers', str(exp['freeze']),
        '--weight_decay', str(exp['wd']),
        '--margin', str(exp['margin']),
        '--batch_size', str(exp.get('batch_size', FIXED_PARAMS['batch_size'])),
        '--grad_accum', str(exp.get('grad_accum', FIXED_PARAMS['grad_accum'])),
        '--epochs', str(FIXED_PARAMS['epochs']),
        '--patience', str(FIXED_PARAMS['patience']),
    ]

    run_start = datetime.now()
    generated_model_file = None # Pour stocker le nom du fichier créé

    try:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )

        output_lines = []
        for line in process.stdout:
            print(line, end='')
            output_lines.append(line)
            # On guette le nom du fichier sauvegardé en local
            if "Saved: " in line:
                # Ex: "Saved: dual_lrGNN...pt"
                try:
                    generated_model_file = line.split("Saved: ")[1].strip()
                except:
                    pass

        process.wait()
        return_code = process.returncode

        # Parsing des résultats (MRR, etc.)
        best_mrr = 0.0
        best_r1 = 0.0
        best_r5 = 0.0

        for line in output_lines:
            if "New Best MRR:" in line:
                try:
                    mrr_str = line.split("New Best MRR:")[1].split("|")[0].strip()
                    best_mrr = float(mrr_str)
                except: pass
            elif "Val MRR:" in line and "R@1:" in line:
                try:
                    parts = line.split("|")
                    for part in parts:
                        if "Val MRR:" in part: best_mrr = max(best_mrr, float(part.split(":")[1].strip()))
                        elif "R@1:" in part: best_r1 = max(best_r1, float(part.split(":")[1].strip()))
                        elif "R@5:" in part: best_r5 = max(best_r5, float(part.split(":")[1].strip()))
                except: pass

        elapsed = (datetime.now() - run_start).total_seconds() / 60
        status = "✅ SUCCESS" if return_code == 0 else "❌ FAILED"

        # === SAUVEGARDE SUR DRIVE ===
        # Si un fichier .pt a été créé, on le déplace sur le Drive
        if generated_model_file and return_code == 0:
            local_path = Path(FIXED_PARAMS['data_dir']) / generated_model_file
            if local_path.exists():
                drive_path = DRIVE_OUTPUT_DIR / generated_model_file
                print(f"📦 Copie vers Drive : {drive_path} ...")
                shutil.copy2(local_path, drive_path)
                print("✅ Copie terminée.")
            else:
                print(f"⚠️ Fichier local introuvable : {local_path}")

    except Exception as e:
        print(f"\n❌ ERREUR : {e}")
        elapsed = (datetime.now() - run_start).total_seconds() / 60
        status = f"❌ ERROR: {str(e)[:50]}"
        return_code = -1

    # Enregistrement des résultats
    result = {
        'Run': i,
        'Name': exp['name'],
        'LR_GNN': exp['lr_gnn'],
        'LR_BERT': exp['lr_bert'],
        'Freeze_Layers': exp['freeze'],
        'Weight_Decay': exp['wd'],
        'Margin': exp['margin'],
        'MRR': best_mrr,
        'R@1': best_r1,
        'R@5': best_r5,
        'Time_min': elapsed,
        'Status': status,
        'Model_File': generated_model_file if generated_model_file else "N/A"
    }

    # Mise à jour de la liste et sauvegarde CSV immédiate sur le Drive
    results.append(result)
    df_temp = pd.DataFrame(results)
    df_temp.to_csv(RESULTS_CSV, index=False)
    print(f"\n💾 CSV mis à jour sur Drive : {RESULTS_CSV}")

    # Progression
    total_elapsed = (datetime.now() - total_start).total_seconds() / 60
    print(f"\n⏱️  Temps écoulé : {total_elapsed:.1f}min")

    if i < len(EXPERIMENTS):
        print("\n⏸️  Pause de 30s...")
        time.sleep(30)

print("\n🎉 GRID SEARCH TERMINÉ - TOUT EST SUR LE DRIVE !")

🔄 Connexion au Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dossier de sauvegarde sécurisé : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder
🚀 DUAL ENCODER - GRID SEARCH AUTOMATIQUE (DRIVE SAVE MODE)
Dossier de sortie    : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder
Fichier de résultats : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder/dual_encoder_results.csv
Nombre d'expériences : 15
--------------------------------------------------------------------------------
🔄 Reprise du fichier existant : /content/drive/MyDrive/Altegrad_Results/GridSearch_DualEncoder/dual_encoder_results.csv

RUN 1/15 : Freeze 6 Layers
LR GNN      : 0.0008
LR BERT     : 3e-05
Freeze      : 6 layers
Weight Decay: 0.0001
Margin      : 0.2
🚀 DUAL ENCODER TRAINING - OPTIMIZED (AMP FIX)
LR GNN      : 0.0008
LR BERT     : 3e-05
Weight Decay: 0.0001
Freeze Layers: 6
Mar

## other code analyse

In [ ]:
import pandas as pd
import numpy as np

# =========================================================
# ANALYSE DES RÉSULTATS DUAL ENCODER
# =========================================================
csv_path = 'data_baseline/dual_encoder_results.csv'

df = pd.read_csv(csv_path)
df_sorted = df.sort_values(by='MRR', ascending=False)

print("=" * 80)
print("📊 ANALYSE DÉTAILLÉE - DUAL ENCODER")
print("=" * 80)

# Statistiques globales
print(f"\nNombre d'expériences : {len(df)}")
print(f"Runs réussis         : {sum('✅' in str(s) for s in df['Status'])}")
print(f"Temps total          : {df['Time_min'].sum() / 60:.1f}h")
print(f"Temps moyen/run      : {df['Time_min'].mean():.1f}min")

print(f"\n📈 STATISTIQUES MRR :")
print(f"   Moyenne : {df['MRR'].mean():.4f}")
print(f"   Médiane : {df['MRR'].median():.4f}")
print(f"   Min     : {df['MRR'].min():.4f}")
print(f"   Max     : {df['MRR'].max():.4f}")
print(f"   Std     : {df['MRR'].std():.4f}")

# TOP 10
print("\n🏆 TOP 10 CONFIGURATIONS :")
print("-" * 80)
cols = ['Run', 'Name', 'LR_BERT', 'Freeze_Layers', 'Weight_Decay', 'Margin', 'MRR', 'R@1']
print(df_sorted[cols].head(10).to_string(index=False))

# MEILLEURE PAR CATÉGORIE
print("\n" + "=" * 80)
print("🥇 MEILLEURES CONFIGS PAR CATÉGORIE")
print("=" * 80)

# Meilleure Full Training
df_full = df[df['Freeze_Layers'] == 0].sort_values('MRR', ascending=False)
if len(df_full) > 0:
    best_full = df_full.iloc[0]
    print(f"\n1. FULL TRAINING (Freeze=0)")
    print(f"   Config : {best_full['Name']}")
    print(f"   LR BERT: {best_full['LR_BERT']}")
    print(f"   MRR    : {best_full['MRR']:.4f}")

# Meilleure Freeze
df_freeze = df[df['Freeze_Layers'] > 0].sort_values('MRR', ascending=False)
if len(df_freeze) > 0:
    best_freeze = df_freeze.iloc[0]
    print(f"\n2. FROZEN LAYERS (Freeze>0)")
    print(f"   Config : {best_freeze['Name']}")
    print(f"   Freeze : {best_freeze['Freeze_Layers']} layers")
    print(f"   MRR    : {best_freeze['MRR']:.4f}")

    # Comparaison Full vs Freeze
    if len(df_full) > 0:
        print(f"\n   📊 Full vs Freeze :")
        print(f"      Full : MRR {best_full['MRR']:.4f} | Temps {best_full['Time_min']:.1f}min")
        print(f"      Freeze: MRR {best_freeze['MRR']:.4f} | Temps {best_freeze['Time_min']:.1f}min")
        print(f"      Gain MRR   : {(best_full['MRR'] - best_freeze['MRR']):+.4f}")
        print(f"      Gain Temps : {(best_freeze['Time_min'] - best_full['Time_min']):.1f}min")

# Heatmap LR BERT vs Freeze
print("\n" + "=" * 80)
print("🔥 HEATMAP : LR_BERT × FREEZE_LAYERS (MRR moyen)")
print("=" * 80)
heatmap = df.pivot_table(values='MRR', index='LR_BERT', columns='Freeze_Layers', aggfunc='mean')
print(heatmap.to_string())

# Recommandations
print("\n" + "=" * 80)
print("💡 RECOMMANDATIONS")
print("=" * 80)

best_overall = df_sorted.iloc[0]
print(f"\n1. MEILLEURE CONFIG GLOBALE :")
print(f"   → {best_overall['Name']}")
print(f"   → MRR : {best_overall['MRR']:.4f}")

# Meilleur compromis temps/perf
df['efficiency'] = df['MRR'] / (df['Time_min'] / 60)  # MRR par heure
best_efficient = df.sort_values('efficiency', ascending=False).iloc[0]
print(f"\n2. MEILLEUR COMPROMIS TEMPS/PERFORMANCE :")
print(f"   → {best_efficient['Name']}")
print(f"   → MRR : {best_efficient['MRR']:.4f} en {best_efficient['Time_min']:.1f}min")
print(f"   → Efficiency : {best_efficient['efficiency']:.4f} MRR/h")

print("\n" + "=" * 80)

In [ ]:
!python -m pip -q install -U evaluate bert-score sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.9 MB/s eta 0:00:00


# autre test à check si les suivant marchent pas

In [ ]:
def load_dual_encoder(checkpoint_path, device='cuda'):
    """
    Charge un DualEncoder depuis un checkpoint (gère nouveau et ancien format).

    Args:
        checkpoint_path: Chemin vers le fichier .pt
        device: 'cuda' ou 'cpu'

    Returns:
        model: DualEncoder chargé
        config: Dictionnaire de config (si disponible)
    """
    from data_baseline.train_dual_encoder_optimized import DualEncoder

    checkpoint_path = Path(checkpoint_path)
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # === FORMAT DETECTION ===
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        # ✅ NOUVEAU FORMAT (avec metadata)
        print(f"✅ Nouveau format détecté")
        state_dict = checkpoint['model_state_dict']

        # Extrait config si disponible
        if 'args' in checkpoint:
            args = checkpoint['args']
            gnn_config = {
                'hidden_dim': args.get('hidden_dim', 256),
                'out_dim': 768,
                'num_layers': args.get('num_layers', 4),
                'num_heads': args.get('num_heads', 4),
                'dropout': args.get('dropout', 0.1),
            }
            freeze_layers = args.get('freeze_layers', 0)
            model_name = args.get('model_name', 'recobo/chemical-bert-uncased')
        else:
            # Config par défaut
            gnn_config = {
                'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4,
                'num_heads': 4, 'dropout': 0.1
            }
            freeze_layers = 0
            model_name = 'recobo/chemical-bert-uncased'

        print(f"   Config GNN : {gnn_config}")
        print(f"   Freeze     : {freeze_layers} layers")

    elif isinstance(checkpoint, dict) and 'config' in checkpoint:
        # ✅ AUTRE NOUVEAU FORMAT (avec 'config' directement)
        print(f"✅ Nouveau format (avec config) détecté")
        state_dict = checkpoint['model_state_dict']
        config = checkpoint['config']
        # Adapter selon ta structure
        gnn_config = config.get('gnn_config', {})
        freeze_layers = config.get('freeze_layers', 0)
        model_name = config.get('model_name', 'recobo/chemical-bert-uncased')

    else:
        # ❌ ANCIEN FORMAT (juste state_dict)
        print(f"⚠️  Ancien format détecté (pas de metadata)")
        state_dict = checkpoint

        # Config par défaut (à ajuster selon tes besoins)
        gnn_config = {
            'hidden_dim': 256,
            'out_dim': 768,
            'num_layers': 4,
            'num_heads': 4,
            'dropout': 0.1,
        }
        freeze_layers = 0
        model_name = 'recobo/chemical-bert-uncased'

        print(f"   ⚠️  Utilisation de config par défaut : {gnn_config}")

    # === CRÉATION DU MODÈLE ===
    model = DualEncoder(
        model_name=model_name,
        gnn_args=gnn_config,
        freeze_layers=freeze_layers
    )

    # === CHARGEMENT DES POIDS ===
    try:
        model.load_state_dict(state_dict, strict=True)
        print(f"✅ Modèle chargé avec succès (strict=True)")
    except RuntimeError as e:
        print(f"⚠️  Chargement strict échoué, essai non-strict...")
        model.load_state_dict(state_dict, strict=False)
        print(f"⚠️  Modèle chargé (strict=False) - Certains poids peuvent manquer")

    model.to(device)
    model.eval()

    return model, gnn_config


# === UTILISATION ===
checkpoint_path = "data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt"
model, config = load_dual_encoder(checkpoint_path, device='cuda')

print(f"\n📊 Modèle prêt pour évaluation")
print(f"   Config : {config}")

✅ Nouveau format détecté
   Config GNN : {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}
   Freeze     : 0 layers


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modèle chargé avec succès (strict=True)

📊 Modèle prêt pour évaluation
   Config : {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pathlib import Path
from data_baseline.train_dual_encoder_optimized import DualEncoder, RawTextGraphDataset, DualCollate
from transformers import AutoTokenizer

def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge un DualEncoder (gère nouveau et ancien format)."""
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Détection du format
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        # Nouveau format
        state_dict = checkpoint['model_state_dict']
        args = checkpoint.get('args', {})

        gnn_config = {
            'hidden_dim': args.get('hidden_dim', 256),
            'out_dim': 768,
            'num_layers': args.get('num_layers', 4),
            'num_heads': args.get('num_heads', 4),
            'dropout': args.get('dropout', 0.1),
        }
        freeze_layers = args.get('freeze_layers', 0)
        model_name = args.get('model_name', 'recobo/chemical-bert-uncased')

        print(f"✅ Nouveau format - Config: {gnn_config}, Freeze: {freeze_layers}")
    else:
        # Ancien format
        state_dict = checkpoint
        gnn_config = {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}
        freeze_layers = 0
        model_name = 'recobo/chemical-bert-uncased'
        print(f"⚠️  Ancien format - Config par défaut")

    # Création et chargement
    model = DualEncoder(model_name, gnn_config, freeze_layers)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    return model, gnn_config


@torch.no_grad()
def evaluate_dual_encoder(model, val_loader, device):
    """Évalue le MRR sur le validation set."""
    all_g_emb, all_t_emb = [], []

    for graphs, text_inputs in val_loader:
        graphs = graphs.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

        g_emb, t_emb = model(graphs, text_inputs)
        all_g_emb.append(g_emb)
        all_t_emb.append(t_emb)

    all_g_emb = torch.cat(all_g_emb, dim=0)
    all_t_emb = torch.cat(all_t_emb, dim=0)

    # Text-to-Molecule retrieval
    sims = all_t_emb @ all_g_emb.t()
    ranks = sims.argsort(dim=-1, descending=True)

    N = sims.size(0)
    correct = torch.arange(N, device=device)
    positions = (ranks == correct.unsqueeze(1)).nonzero()[:, 1] + 1

    mrr = (1.0 / positions.float()).mean().item()
    r1 = (positions <= 1).float().mean().item()
    r5 = (positions <= 5).float().mean().item()
    r10 = (positions <= 10).float().mean().item()

    return {'MRR': mrr, 'R@1': r1, 'R@5': r5, 'R@10': r10}


def run_evaluation():
    """Fonction principale d'évaluation."""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"⚙️  Configuration : {device}")

    # Chemins
    checkpoint_path = "data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt"
    val_data_path = "data_baseline/data/validation_graphs.pkl"

    print(f"📥 Chargement du Dual Encoder depuis {checkpoint_path}")

    # Charge le modèle
    model, config = load_dual_encoder(checkpoint_path, device)

    # Prépare le DataLoader
    tokenizer = AutoTokenizer.from_pretrained('recobo/chemical-bert-uncased')
    val_dataset = RawTextGraphDataset(val_data_path)
    val_loader = DataLoader(
        val_dataset,
        batch_size=64,
        shuffle=False,
        collate_fn=DualCollate(tokenizer),
        num_workers=2
    )

    print(f"📊 Évaluation sur {len(val_dataset)} exemples...")

    # Évaluation
    metrics = evaluate_dual_encoder(model, val_loader, device)

    print("\n" + "=" * 60)
    print("🏆 RÉSULTATS DUAL ENCODER")
    print("=" * 60)
    print(f"MRR  : {metrics['MRR']:.4f}")
    print(f"R@1  : {metrics['R@1']:.4f}")
    print(f"R@5  : {metrics['R@5']:.4f}")
    print(f"R@10 : {metrics['R@10']:.4f}")
    print("=" * 60)

    return metrics


if __name__ == "__main__":
    run_evaluation()

⚙️  Configuration : cuda
📥 Chargement du Dual Encoder depuis data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Nouveau format - Config: {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}, Freeze: 0
📊 Évaluation sur 1000 exemples...

🏆 RÉSULTATS DUAL ENCODER
MRR  : 0.9398
R@1  : 0.9030
R@5  : 0.9850
R@10 : 0.9920


In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
MODEL_PATH = "data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        # Freeze layers si nécessaire
        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder (gère nouveau format)."""
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Détection format
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        # ✅ Nouveau format
        state_dict = checkpoint['model_state_dict']
        args = checkpoint.get('args', {})

        gnn_config = {
            'hidden_dim': args.get('hidden_dim', 256),
            'out_dim': 768,
            'num_layers': args.get('num_layers', 4),
            'num_heads': args.get('num_heads', 4),
            'dropout': args.get('dropout', 0.1),
        }
        freeze_layers = args.get('freeze_layers', 0)
        model_name = args.get('model_name', MODEL_NAME)

        print(f"✅ Nouveau format - Config: {gnn_config}, Freeze: {freeze_layers}")
    else:
        # ❌ Ancien format
        state_dict = checkpoint
        gnn_config = {
            'hidden_dim': 256,
            'out_dim': 768,
            'num_layers': 4,
            'num_heads': 4,
            'dropout': 0.1
        }
        freeze_layers = 0
        model_name = MODEL_NAME
        print(f"⚠️ Ancien format - Config par défaut")

    # Création modèle
    model = DualEncoder(model_name, gnn_config, freeze_layers)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    return model, gnn_config


def load_texts_and_graphs(pkl_path):
    """Charge les graphes et extrait les descriptions."""
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    texts = [d.description for d in data]
    return data, texts


class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx]


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings du TRAIN avec le BERT fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes) avec le modèle fine-tuné...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Train Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def run_evaluation():
    """Évaluation complète avec BLEU-4 et BERTScore."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN) ===
    # Encode tous les textes du train avec le BERT fine-tuné
    train_embs = generate_reference_embeddings(
        model, tokenizer, train_texts, DEVICE, batch_size=BATCH_SIZE
    )
    train_embs = train_embs.to(DEVICE)  # [N_train, 768]

    print(f"   Embeddings train: {train_embs.shape}")

    # === 4. ENCODAGE GRAPHES VALIDATION ===
    val_mol_embs = encode_graphs(model, val_graphs, DEVICE, batch_size=BATCH_SIZE)
    val_mol_embs = val_mol_embs.to(DEVICE)  # [N_val, 768]

    print(f"   Embeddings val:   {val_mol_embs.shape}")

    # === 5. RETRIEVAL (Recherche du plus proche voisin) ===
    print("🔍 Recherche des descriptions les plus proches dans le TRAIN...")

    # Similarité cosine : [N_val, N_train]
    sims = val_mol_embs @ train_embs.t()

    # Meilleur match pour chaque graphe de validation
    best_indices = sims.argmax(dim=-1).cpu().numpy()

    # Récupération des textes prédits
    preds = [train_texts[idx] for idx in best_indices]
    refs = val_texts

    # === 6. CALCUL DES MÉTRIQUES NLP ===
    print("📊 Calcul des scores NLP (BLEU-4 & BERTScore)...")

    # BLEU-4
    bleu_metric = evaluate.load("bleu")
    bleu_res = bleu_metric.compute(
        predictions=preds,
        references=[[r] for r in refs],  # Format attendu : liste de listes
        max_order=4
    )
    bleu4 = bleu_res["bleu"]

    # BERTScore
    bert_metric = evaluate.load("bertscore")
    bert_res = bert_metric.compute(
        predictions=preds,
        references=refs,
        model_type="roberta-base",
        lang="en"
    )
    bert_f1 = float(np.mean(bert_res["f1"]))

    # Score final (comme dans ta formule)
    final_proxy = 0.5 * (bleu4 if bleu4 <= 1.0 else bleu4/100) + 0.5 * bert_f1

    # === 7. AFFICHAGE RÉSULTATS ===
    print("\n" + "=" * 70)
    print("🏆 RÉSULTATS DUAL ENCODER (NLP METRICS)")
    print("=" * 70)
    print(f"BLEU-4        : {bleu4:.5f}")
    print(f"BERTScore F1  : {bert_f1:.5f}")
    print(f"Final Score   : {final_proxy:.5f}")
    print(f"Kaggle Score  : {0.925 * final_proxy:.5f}")
    print("=" * 70)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS :")
    print("-" * 70)
    for i in range(min(3, len(preds))):
        print(f"\nExemple {i+1}:")
        print(f"  Prédit  : {preds[i][:100]}...")
        print(f"  Référence: {refs[i][:100]}...")
    print("-" * 70)

    return {
        'bleu4': bleu4,
        'bertscore_f1': bert_f1,
        'final_score': final_proxy,
        'kaggle_score': 0.925 * final_proxy
    }


if __name__ == "__main__":
    results = run_evaluation()

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt
✅ Nouveau format - Config: {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}, Freeze: 0


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
🔄 Recalcul des embeddings de référence (31008 textes) avec le modèle fine-tuné...


Encoding Train Texts: 100%|██████████| 969/969 [01:40<00:00,  9.65it/s]


   Embeddings train: torch.Size([31008, 768])
🧪 Encodage de 1000 graphes...


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 RÉSULTATS DUAL ENCODER (NLP METRICS)
BLEU-4        : 0.43530
BERTScore F1  : 0.92371
Final Score   : 0.67950
Kaggle Score  : 0.62854

📝 EXEMPLES DE PRÉDICTIONS :
----------------------------------------------------------------------

Exemple 1:
  Prédit  : The molecule is an aminotetrasaccharide consisting of two units of 2-acetamido-2-deoxy-4-O-sulfo-bet...
  Référence: The molecule is an amino tetrasaccharide comprised of an L-iduronic acid residue, sulfated on O-2, a...

Exemple 2:
  Prédit  : The molecule is the D-enantiomer of the alpha-amino acid lysine. It has a role as a bacterial metabo...
  Référence: The molecule is the non-proteinogenic L-alpha-amino acid that is norspermidine (1,5,9-triazanonane) ...

Exemple 3:
  Prédit  : The molecule is an (omega-1)-hydroxy fatty acid ascaroside obtained by formal condensation of the al...
  Référence: The molecule is an (omega-1)-hydroxy fatty acid ascaroside obtained by formal condensation of the al...
-----------------------------

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
MODEL_PATH = "data_baseline/data/dual_lrGNN0.0008_lrBERT1e-05_wd0.0001_frz0_margin0.2_bs128.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        # Freeze layers si nécessaire
        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder (gère nouveau format)."""
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Détection format
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        # ✅ Nouveau format
        state_dict = checkpoint['model_state_dict']
        args = checkpoint.get('args', {})

        gnn_config = {
            'hidden_dim': args.get('hidden_dim', 256),
            'out_dim': 768,
            'num_layers': args.get('num_layers', 4),
            'num_heads': args.get('num_heads', 4),
            'dropout': args.get('dropout', 0.1),
        }
        freeze_layers = args.get('freeze_layers', 0)
        model_name = args.get('model_name', MODEL_NAME)

        print(f"✅ Nouveau format - Config: {gnn_config}, Freeze: {freeze_layers}")
    else:
        # ❌ Ancien format
        state_dict = checkpoint
        gnn_config = {
            'hidden_dim': 256,
            'out_dim': 768,
            'num_layers': 4,
            'num_heads': 4,
            'dropout': 0.1
        }
        freeze_layers = 0
        model_name = MODEL_NAME
        print(f"⚠️ Ancien format - Config par défaut")

    # Création modèle
    model = DualEncoder(model_name, gnn_config, freeze_layers)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    return model, gnn_config


def load_texts_and_graphs(pkl_path):
    """Charge les graphes et extrait les descriptions."""
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    texts = [d.description for d in data]
    return data, texts


class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx]


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings du TRAIN avec le BERT fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes) avec le modèle fine-tuné...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Train Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def run_evaluation():
    """Évaluation complète avec BLEU-4 et BERTScore."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN) ===
    # Encode tous les textes du train avec le BERT fine-tuné
    train_embs = generate_reference_embeddings(
        model, tokenizer, train_texts, DEVICE, batch_size=BATCH_SIZE
    )
    train_embs = train_embs.to(DEVICE)  # [N_train, 768]

    print(f"   Embeddings train: {train_embs.shape}")

    # === 4. ENCODAGE GRAPHES VALIDATION ===
    val_mol_embs = encode_graphs(model, val_graphs, DEVICE, batch_size=BATCH_SIZE)
    val_mol_embs = val_mol_embs.to(DEVICE)  # [N_val, 768]

    print(f"   Embeddings val:   {val_mol_embs.shape}")

    # === 5. RETRIEVAL (Recherche du plus proche voisin) ===
    print("🔍 Recherche des descriptions les plus proches dans le TRAIN...")

    # Similarité cosine : [N_val, N_train]
    sims = val_mol_embs @ train_embs.t()

    # Meilleur match pour chaque graphe de validation
    best_indices = sims.argmax(dim=-1).cpu().numpy()

    # Récupération des textes prédits
    preds = [train_texts[idx] for idx in best_indices]
    refs = val_texts

    # === 6. CALCUL DES MÉTRIQUES NLP ===
    print("📊 Calcul des scores NLP (BLEU-4 & BERTScore)...")

    # BLEU-4
    bleu_metric = evaluate.load("bleu")
    bleu_res = bleu_metric.compute(
        predictions=preds,
        references=[[r] for r in refs],  # Format attendu : liste de listes
        max_order=4
    )
    bleu4 = bleu_res["bleu"]

    # BERTScore
    bert_metric = evaluate.load("bertscore")
    bert_res = bert_metric.compute(
        predictions=preds,
        references=refs,
        model_type="roberta-base",
        lang="en"
    )
    bert_f1 = float(np.mean(bert_res["f1"]))

    # Score final (comme dans ta formule)
    final_proxy = 0.5 * (bleu4 if bleu4 <= 1.0 else bleu4/100) + 0.5 * bert_f1

    # === 7. AFFICHAGE RÉSULTATS ===
    print("\n" + "=" * 70)
    print("🏆 RÉSULTATS DUAL ENCODER (NLP METRICS)")
    print("=" * 70)
    print(f"BLEU-4        : {bleu4:.5f}")
    print(f"BERTScore F1  : {bert_f1:.5f}")
    print(f"Final Score   : {final_proxy:.5f}")
    print(f"Kaggle Score  : {0.925 * final_proxy:.5f}")
    print("=" * 70)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS :")
    print("-" * 70)
    for i in range(min(3, len(preds))):
        print(f"\nExemple {i+1}:")
        print(f"  Prédit  : {preds[i][:100]}...")
        print(f"  Référence: {refs[i][:100]}...")
    print("-" * 70)

    return {
        'bleu4': bleu4,
        'bertscore_f1': bert_f1,
        'final_score': final_proxy,
        'kaggle_score': 0.925 * final_proxy
    }


if __name__ == "__main__":
    results = run_evaluation()

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis data_baseline/data/dual_lrGNN0.0008_lrBERT1e-05_wd0.0001_frz0_margin0.2_bs128.pt
✅ Nouveau format - Config: {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}, Freeze: 0


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
🔄 Recalcul des embeddings de référence (31008 textes) avec le modèle fine-tuné...


Encoding Train Texts: 100%|██████████| 969/969 [01:40<00:00,  9.68it/s]


   Embeddings train: torch.Size([31008, 768])
🧪 Encodage de 1000 graphes...


Encoding Graphs: 100%|██████████| 32/32 [00:00<00:00, 74.83it/s]


   Embeddings val:   torch.Size([1000, 768])
🔍 Recherche des descriptions les plus proches dans le TRAIN...
📊 Calcul des scores NLP (BLEU-4 & BERTScore)...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 RÉSULTATS DUAL ENCODER (NLP METRICS)
BLEU-4        : 0.42114
BERTScore F1  : 0.92167
Final Score   : 0.67141
Kaggle Score  : 0.62105

📝 EXEMPLES DE PRÉDICTIONS :
----------------------------------------------------------------------

Exemple 1:
  Prédit  : The molecule is an aminotetrasaccharide consisting of two units of 2-acetamido-2-deoxy-4-O-sulfo-bet...
  Référence: The molecule is an amino tetrasaccharide comprised of an L-iduronic acid residue, sulfated on O-2, a...

Exemple 2:
  Prédit  : The molecule is the D-enantiomer of the alpha-amino acid lysine. It has a role as a bacterial metabo...
  Référence: The molecule is the non-proteinogenic L-alpha-amino acid that is norspermidine (1,5,9-triazanonane) ...

Exemple 3:
  Prédit  : The molecule is an (omega-1)-hydroxy fatty acid ascaroside obtained by formal condensation of the al...
  Référence: The molecule is an (omega-1)-hydroxy fatty acid ascaroside obtained by formal condensation of the al...
-----------------------------

# Code pour avoir les résultats et convertir au format csv pour kaggle

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
MODEL_PATH = "data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Output
SUBMISSION_PATH = "submission_dual_encoder.csv"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder (gère nouveau format)."""
    checkpoint = torch.load(checkpoint_path, map_location=device)

    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
        args = checkpoint.get('args', {})

        gnn_config = {
            'hidden_dim': args.get('hidden_dim', 256),
            'out_dim': 768,
            'num_layers': args.get('num_layers', 4),
            'num_heads': args.get('num_heads', 4),
            'dropout': args.get('dropout', 0.1),
        }
        freeze_layers = args.get('freeze_layers', 0)
        model_name = args.get('model_name', MODEL_NAME)

        print(f"✅ Nouveau format - Config: {gnn_config}, Freeze: {freeze_layers}")
    else:
        state_dict = checkpoint
        gnn_config = {
            'hidden_dim': 256,
            'out_dim': 768,
            'num_layers': 4,
            'num_heads': 4,
            'dropout': 0.1
        }
        freeze_layers = 0
        model_name = MODEL_NAME
        print(f"⚠️ Ancien format - Config par défaut")

    model = DualEncoder(model_name, gnn_config, freeze_layers)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    return model, gnn_config


def load_texts_and_graphs(pkl_path, has_description=True):
    """
    Charge les graphes et extrait les descriptions (si disponibles).

    Args:
        pkl_path: Chemin vers le fichier pickle
        has_description: Si False, retourne None pour les textes
    """
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    if has_description:
        texts = [d.description for d in data]
    else:
        texts = None  # Pas de description dans le test set

    return data, texts


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings du TRAIN avec le BERT fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes)...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Train Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


def generate_submission(predictions, output_path):
    """
    Génère le fichier de submission Kaggle.

    Args:
        predictions: Liste de textes prédits (dans l'ordre du test set)
        output_path: Chemin du fichier CSV à créer
    """
    submission_df = pd.DataFrame({
        'ID': range(len(predictions)),
        'description': predictions
    })

    submission_df.to_csv(output_path, index=False)
    print(f"\n💾 Submission sauvegardée : {output_path}")
    print(f"   Nombre de prédictions : {len(predictions)}")
    print(f"   Preview :")
    print(submission_df.head(10).to_string(index=False))


@torch.no_grad()
def run_evaluation_and_submission():
    """Évaluation + Génération de submission Kaggle."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")

    # Test set pour Kaggle
    # APRÈS
    test_graphs, _ = load_texts_and_graphs(DATA_DIR / "test_graphs.pkl", has_description=False)


    # Validation set (pour métriques locales)
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")
    print(f"   Test:  {len(test_graphs)} exemples (pour Kaggle)")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN) ===
    train_embs = generate_reference_embeddings(
        model, tokenizer, train_texts, DEVICE, batch_size=BATCH_SIZE
    )
    train_embs = train_embs.to(DEVICE)

    print(f"   Embeddings train: {train_embs.shape}")

    # === 4A. ENCODAGE GRAPHES VALIDATION (pour métriques) ===
    print("\n" + "=" * 70)
    print("📊 ÉVALUATION SUR VALIDATION SET")
    print("=" * 70)

    val_mol_embs = encode_graphs(model, val_graphs, DEVICE, batch_size=BATCH_SIZE)
    val_mol_embs = val_mol_embs.to(DEVICE)

    # Retrieval validation
    print("🔍 Recherche des descriptions les plus proches (Validation)...")
    sims_val = val_mol_embs @ train_embs.t()
    best_indices_val = sims_val.argmax(dim=-1).cpu().numpy()

    preds_val = [train_texts[idx] for idx in best_indices_val]
    refs_val = val_texts

    # Calcul métriques
    print("📊 Calcul des scores NLP (BLEU-4 & BERTScore)...")

    bleu_metric = evaluate.load("bleu")
    bleu_res = bleu_metric.compute(
        predictions=preds_val,
        references=[[r] for r in refs_val],
        max_order=4
    )
    bleu4 = bleu_res["bleu"]

    bert_metric = evaluate.load("bertscore")
    bert_res = bert_metric.compute(
        predictions=preds_val,
        references=refs_val,
        model_type="roberta-base",
        lang="en"
    )
    bert_f1 = float(np.mean(bert_res["f1"]))

    final_proxy = 0.5 * (bleu4 if bleu4 <= 1.0 else bleu4/100) + 0.5 * bert_f1

    # Affichage résultats validation
    print("\n" + "=" * 70)
    print("🏆 RÉSULTATS VALIDATION (NLP METRICS)")
    print("=" * 70)
    print(f"BLEU-4        : {bleu4:.5f}")
    print(f"BERTScore F1  : {bert_f1:.5f}")
    print(f"Final Score   : {final_proxy:.5f}")
    print(f"Kaggle Score  : {0.925 * final_proxy:.5f}")
    print("=" * 70)

    # === 4B. GÉNÉRATION PRÉDICTIONS TEST (pour Kaggle) ===
    print("\n" + "=" * 70)
    print("🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)")
    print("=" * 70)

    test_mol_embs = encode_graphs(model, test_graphs, DEVICE, batch_size=BATCH_SIZE)
    test_mol_embs = test_mol_embs.to(DEVICE)

    print("🔍 Recherche des descriptions les plus proches (Test)...")
    sims_test = test_mol_embs @ train_embs.t()
    best_indices_test = sims_test.argmax(dim=-1).cpu().numpy()

    preds_test = [train_texts[idx] for idx in best_indices_test]

    # Génération du fichier submission
    generate_submission(preds_test, SUBMISSION_PATH)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS (TEST SET) :")
    print("-" * 70)
    for i in range(min(5, len(preds_test))):
        print(f"\nId {i}:")
        print(f"  {preds_test[i][:120]}...")
    print("-" * 70)

    return {
        'bleu4': bleu4,
        'bertscore_f1': bert_f1,
        'final_score': final_proxy,
        'kaggle_score': 0.925 * final_proxy,
        'submission_file': SUBMISSION_PATH
    }


if __name__ == "__main__":
    results = run_evaluation_and_submission()

    print("\n" + "=" * 70)
    print("✅ TERMINÉ !")
    print("=" * 70)
    print(f"📄 Fichier de submission : {results['submission_file']}")
    print(f"📊 Score validation      : {results['final_score']:.5f}")
    print(f"🎯 Score Kaggle estimé   : {results['kaggle_score']:.5f}")
    print("=" * 70)
    print("\n💡 Prochaine étape : Upload le fichier sur Kaggle !")

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis data_baseline/data/dual_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128.pt


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Nouveau format - Config: {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}, Freeze: 0
📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
   Test:  1000 exemples (pour Kaggle)
🔄 Recalcul des embeddings de référence (31008 textes)...


Encoding Train Texts: 100%|██████████| 969/969 [01:41<00:00,  9.57it/s]


   Embeddings train: torch.Size([31008, 768])

📊 ÉVALUATION SUR VALIDATION SET
🧪 Encodage de 1000 graphes...


...

🔍 Recherche des descriptions les plus proches (Validation)...
📊 Calcul des scores NLP (BLEU-4 & BERTScore)...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 RÉSULTATS VALIDATION (NLP METRICS)
BLEU-4        : 0.43530
BERTScore F1  : 0.92371
Final Score   : 0.67950
Kaggle Score  : 0.62854

🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)
🧪 Encodage de 1000 graphes...


Encoding Graphs: 100%|██████████| 32/32 [00:00<00:00, 75.54it/s]


🔍 Recherche des descriptions les plus proches (Test)...

💾 Submission sauvegardée : submission_dual_encoder.csv
   Nombre de prédictions : 1000
   Preview :
 ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         description
  0                                                                                                                                                                                                                               The molecule is a beta-D-glucosyl-(1<->1')-N-acylsphinganine in which the acyl group specified is eicosanoyl. It has a role as a 

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
MODEL_PATH = "data_baseline/data/dual_lrGNN0.0008_lrBERT1e-05_wd0.0001_frz0_margin0.2_bs128.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Output
SUBMISSION_PATH = "submission_dual_encoder.csv"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder (gère nouveau format)."""
    checkpoint = torch.load(checkpoint_path, map_location=device)

    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
        args = checkpoint.get('args', {})

        gnn_config = {
            'hidden_dim': args.get('hidden_dim', 256),
            'out_dim': 768,
            'num_layers': args.get('num_layers', 4),
            'num_heads': args.get('num_heads', 4),
            'dropout': args.get('dropout', 0.1),
        }
        freeze_layers = args.get('freeze_layers', 0)
        model_name = args.get('model_name', MODEL_NAME)

        print(f"✅ Nouveau format - Config: {gnn_config}, Freeze: {freeze_layers}")
    else:
        state_dict = checkpoint
        gnn_config = {
            'hidden_dim': 256,
            'out_dim': 768,
            'num_layers': 4,
            'num_heads': 4,
            'dropout': 0.1
        }
        freeze_layers = 0
        model_name = MODEL_NAME
        print(f"⚠️ Ancien format - Config par défaut")

    model = DualEncoder(model_name, gnn_config, freeze_layers)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    return model, gnn_config


def load_texts_and_graphs(pkl_path, has_description=True):
    """
    Charge les graphes et extrait les descriptions (si disponibles).

    Args:
        pkl_path: Chemin vers le fichier pickle
        has_description: Si False, retourne None pour les textes
    """
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    if has_description:
        texts = [d.description for d in data]
    else:
        texts = None  # Pas de description dans le test set

    return data, texts


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings du TRAIN avec le BERT fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes)...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Train Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


def generate_submission(predictions, output_path):

#def generate_submission(predictions, output_path):
    #submission_df = pd.DataFrame({
    #    'ID': range(len(predictions)),      # ← Correction : 'ID' en majuscules
    #    'Predicted': predictions
    #})
    #
    #submission_df.to_csv(output_path, index=False)
    #...
    """
    Génère le fichier de submission Kaggle.

    Args:
        predictions: Liste de textes prédits (dans l'ordre du test set)
        output_path: Chemin du fichier CSV à créer
    """
    submission_df = pd.DataFrame({
        'ID': range(len(predictions)),
        'description': predictions
    })

    submission_df.to_csv(output_path, index=False)
    print(f"\n💾 Submission sauvegardée : {output_path}")
    print(f"   Nombre de prédictions : {len(predictions)}")
    print(f"   Preview :")
    print(submission_df.head(10).to_string(index=False))


@torch.no_grad()
def run_evaluation_and_submission():
    """Évaluation + Génération de submission Kaggle."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")

    # Test set pour Kaggle
    # APRÈS
    test_graphs, _ = load_texts_and_graphs(DATA_DIR / "test_graphs.pkl", has_description=False)


    # Validation set (pour métriques locales)
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")
    print(f"   Test:  {len(test_graphs)} exemples (pour Kaggle)")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN) ===
    train_embs = generate_reference_embeddings(
        model, tokenizer, train_texts, DEVICE, batch_size=BATCH_SIZE
    )
    train_embs = train_embs.to(DEVICE)

    print(f"   Embeddings train: {train_embs.shape}")

    # === 4A. ENCODAGE GRAPHES VALIDATION (pour métriques) ===
    print("\n" + "=" * 70)
    print("📊 ÉVALUATION SUR VALIDATION SET")
    print("=" * 70)

    val_mol_embs = encode_graphs(model, val_graphs, DEVICE, batch_size=BATCH_SIZE)
    val_mol_embs = val_mol_embs.to(DEVICE)

    # Retrieval validation
    print("🔍 Recherche des descriptions les plus proches (Validation)...")
    sims_val = val_mol_embs @ train_embs.t()
    best_indices_val = sims_val.argmax(dim=-1).cpu().numpy()

    preds_val = [train_texts[idx] for idx in best_indices_val]
    refs_val = val_texts

    # Calcul métriques
    print("📊 Calcul des scores NLP (BLEU-4 & BERTScore)...")

    bleu_metric = evaluate.load("bleu")
    bleu_res = bleu_metric.compute(
        predictions=preds_val,
        references=[[r] for r in refs_val],
        max_order=4
    )
    bleu4 = bleu_res["bleu"]

    bert_metric = evaluate.load("bertscore")
    bert_res = bert_metric.compute(
        predictions=preds_val,
        references=refs_val,
        model_type="roberta-base",
        lang="en"
    )
    bert_f1 = float(np.mean(bert_res["f1"]))

    final_proxy = 0.5 * (bleu4 if bleu4 <= 1.0 else bleu4/100) + 0.5 * bert_f1

    # Affichage résultats validation
    print("\n" + "=" * 70)
    print("🏆 RÉSULTATS VALIDATION (NLP METRICS)")
    print("=" * 70)
    print(f"BLEU-4        : {bleu4:.5f}")
    print(f"BERTScore F1  : {bert_f1:.5f}")
    print(f"Final Score   : {final_proxy:.5f}")
    print(f"Kaggle Score  : {0.925 * final_proxy:.5f}")
    print("=" * 70)

    # === 4B. GÉNÉRATION PRÉDICTIONS TEST (pour Kaggle) ===
    print("\n" + "=" * 70)
    print("🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)")
    print("=" * 70)

    test_mol_embs = encode_graphs(model, test_graphs, DEVICE, batch_size=BATCH_SIZE)
    test_mol_embs = test_mol_embs.to(DEVICE)

    print("🔍 Recherche des descriptions les plus proches (Test)...")
    sims_test = test_mol_embs @ train_embs.t()
    best_indices_test = sims_test.argmax(dim=-1).cpu().numpy()

    preds_test = [train_texts[idx] for idx in best_indices_test]

    # Génération du fichier submission
    generate_submission(preds_test, SUBMISSION_PATH)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS (TEST SET) :")
    print("-" * 70)
    for i in range(min(5, len(preds_test))):
        print(f"\nId {i}:")
        print(f"  {preds_test[i][:120]}...")
    print("-" * 70)

    return {
        'bleu4': bleu4,
        'bertscore_f1': bert_f1,
        'final_score': final_proxy,
        'kaggle_score': 0.925 * final_proxy,
        'submission_file': SUBMISSION_PATH
    }


if __name__ == "__main__":
    results = run_evaluation_and_submission()

    print("\n" + "=" * 70)
    print("✅ TERMINÉ !")
    print("=" * 70)
    print(f"📄 Fichier de submission : {results['submission_file']}")
    print(f"📊 Score validation      : {results['final_score']:.5f}")
    print(f"🎯 Score Kaggle estimé   : {results['kaggle_score']:.5f}")
    print("=" * 70)
    print("\n💡 Prochaine étape : Upload le fichier sur Kaggle !")

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis data_baseline/data/dual_lrGNN0.0008_lrBERT1e-05_wd0.0001_frz0_margin0.2_bs128.pt
✅ Nouveau format - Config: {'hidden_dim': 256, 'out_dim': 768, 'num_layers': 4, 'num_heads': 4, 'dropout': 0.1}, Freeze: 0


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
   Test:  1000 exemples (pour Kaggle)
🔄 Recalcul des embeddings de référence (31008 textes)...


Encoding Train Texts: 100%|██████████| 969/969 [01:38<00:00,  9.85it/s]


   Embeddings train: torch.Size([31008, 768])

📊 ÉVALUATION SUR VALIDATION SET
🧪 Encodage de 1000 graphes...


...

🔍 Recherche des descriptions les plus proches (Validation)...
📊 Calcul des scores NLP (BLEU-4 & BERTScore)...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 RÉSULTATS VALIDATION (NLP METRICS)
BLEU-4        : 0.42114
BERTScore F1  : 0.92167
Final Score   : 0.67141
Kaggle Score  : 0.62105

🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)
🧪 Encodage de 1000 graphes...


Encoding Graphs: 100%|██████████| 32/32 [00:00<00:00, 76.96it/s]


🔍 Recherche des descriptions les plus proches (Test)...

💾 Submission sauvegardée : submission_dual_encoder.csv
   Nombre de prédictions : 1000
   Preview :
 ID                                                                                                                                                                                                                                                                                                                           description
  0                                                                                                                                               The molecule is a beta-D-galactosyl-(1<->1')-N-acylsphinganine in which the acyl group specified is docosanoyl. It has a role as a mouse metabolite. It derives from a docosanoic acid.
  1                                            The molecule is an organic chloride salt of trospium. It is an antispasmodic drug used for the treatment of overactive bladder. It has a

_____